# Database Operations

In [1]:
# ref: https://stackoverflow.com/questions/23103962/how-to-write-dataframe-to-postgres-table
# ref: https://www.youtube.com/watch?v=77IVf0zgmwI

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import csv
from io import StringIO
from sqlalchemy import create_engine

In [3]:
df = pd.read_csv("youtube-data.csv")
df.fillna(value="n/a", inplace=True)
df['added_date'] = pd.to_datetime(df['added_date'], infer_datetime_format=True)

In [4]:
def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

In [5]:
engine = create_engine('postgresql://<username>:<password>@<hostname>:<port>/<database>')
df.to_sql('videos', engine, method=psql_insert_copy)